In [2]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta

In [ ]:
# Load the Netflix dataset
try:
    df = pd.read_csv('netflix_titles.csv')
    print("Dataset loaded successfully!")
except FileNotFoundError:
    print("Error: netflix_titles.csv file not found.")
    exit()

In [9]:
# Create SQLite database and connection
conn = sqlite3.connect(':memory:')
df.to_sql('netflix', conn, index=False, if_exists='replace')

def run_query(query):
    """Execute SQL query and return results as a DataFrame"""
    return pd.read_sql_query(query, conn)

In [11]:
# Netflix Project Analysis

# 1. Basic table operations
print("\n1. Basic table operations:")
print(run_query("SELECT * FROM netflix LIMIT 5"))
print(run_query("SELECT COUNT(*) as total_content FROM netflix"))
print(run_query("SELECT DISTINCT type FROM netflix"))


1. Basic table operations:
  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water             None   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans             None   
4      s5  TV Show           Kota Factory             None   

                                                cast        country  \
0                                               None  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...           None   
3                                               None           None   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2

In [15]:
# 2. Business Problems & Solutions

In [17]:
# Problem 1: Count the number of Movies vs TV Shows
print("\n1. Count the number of Movies vs TV Shows:")
print(run_query("""
    SELECT
        type,
        COUNT(*) as total_content
    FROM netflix
    GROUP BY type
"""))


1. Count the number of Movies vs TV Shows:
      type  total_content
0    Movie           6131
1  TV Show           2676


In [19]:
# Problem 2: Find the most common rating for movies and TV shows
print("\n2. Most common rating for movies and TV shows:")
print(run_query("""
    SELECT
        type,
        rating
    FROM
    (
       SELECT
          type,
          rating,
          COUNT(*),
          RANK() OVER(PARTITION BY type ORDER BY COUNT(*) DESC) as ranking 
      FROM netflix
      GROUP BY type, rating
      ) as t1
      WHERE ranking=1
"""))


2. Most common rating for movies and TV shows:
      type rating
0    Movie  TV-MA
1  TV Show  TV-MA


In [21]:
# Problem 3: List all movies released in a specific year (2020)
print("\n3. All movies released in 2020:")
print(run_query("""
    SELECT * FROM netflix
    WHERE 
       type='Movie'
       AND
       release_year=2020
"""))


3. All movies released in 2020:
    show_id   type                                              title  \
0        s1  Movie                               Dick Johnson Is Dead   
1       s17  Movie  Europe's Most Dangerous Man: Otto Skorzeny in ...   
2       s79  Movie                                     Tughlaq Durbar   
3       s85  Movie                               Omo Ghetto: the Saga   
4      s104  Movie                                     Shadow Parties   
..      ...    ...                                                ...   
512   s3047  Movie                      All the Freckles in the World   
513   s3061  Movie                                      Ghost Stories   
514   s5973  Movie                                   #cats_the_mewvie   
515   s7595  Movie                 Norm of the North: Family Vacation   
516   s8100  Movie                                        Straight Up   

                                              director  \
0                               

In [23]:
# Problem 4: Find the top 5 countries with the most content on Netflix
print("\n4. Top 5 countries with most content:")
print(run_query("""
    WITH split_countries AS (
        SELECT 
            show_id,
            TRIM(value) as country
        FROM netflix, 
             json_each('["' || REPLACE(country, ',', '","') || '"]')
        WHERE country IS NOT NULL
    )
    SELECT
        country as new_country,
        COUNT(show_id) as total_content
    FROM split_countries
    GROUP BY country
    ORDER BY total_content DESC
    LIMIT 5
"""))


4. Top 5 countries with most content:
      new_country  total_content
0   United States           3690
1           India           1046
2  United Kingdom            806
3          Canada            445
4          France            393


In [25]:
# Problem 5: Identify the longest movie
print("\n5. Longest movie:")
print(run_query("""
    SELECT * FROM netflix
    WHERE
       type='Movie'
       AND
       duration=(SELECT MAX(duration) FROM netflix WHERE type='Movie')
"""))


5. Longest movie:
    show_id   type                                              title  \
0       s52  Movie  InuYasha the Movie 2: The Castle Beyond the Lo...   
1       s53  Movie  InuYasha the Movie 3: Swords of an Honorable R...   
2      s120  Movie                                     Here and There   
3      s338  Movie                                    Good Luck Chuck   
4      s345  Movie                                          My Girl 2   
..      ...    ...                                                ...   
113   s8442  Movie                 The Nutcracker and the Four Realms   
114   s8498  Movie                                      The Sapphires   
115   s8541  Movie                                         The Tuxedo   
116   s8626  Movie                              Tremors 5:  Bloodline   
117   s8686  Movie                                                VS.   

                          director  \
0                Toshiya Shinohara   
1                Toshiya Shi

In [72]:
# Problem 6: Find content added in the last 5 years
print("\n6. Content added in last 5 years:")
current_date = datetime.now().strftime('%Y-%m-%d')
five_years_ago = (datetime.now() - timedelta(days=5*365)).strftime('%Y-%m-%d')
print(run_query(f"""
    SELECT * FROM netflix
    WHERE date_added IS NOT NULL
    AND date(date_added) >= date('{five_years_ago}')
"""))


6. Content added in last 5 years:
Empty DataFrame
Columns: [show_id, type, title, director, cast, country, date_added, release_year, rating, duration, listed_in, description]
Index: []


In [46]:
# Problem 7: Find all content by director 'Rajiv Chilaka'
print("\n7. All content by director 'Quentin Tarantino':")
print(run_query("""
    SELECT * FROM netflix
    WHERE director LIKE '%Quentin Tarantino%'
"""))


7. All content by director 'Quentin Tarantino':
  show_id     type                                title           director  \
0    s393    Movie                     Django Unchained  Quentin Tarantino   
1   s3894  TV Show  The Hateful Eight: Extended Version  Quentin Tarantino   
2   s5204    Movie                    The Hateful Eight  Quentin Tarantino   
3   s7078    Movie                 Inglourious Basterds  Quentin Tarantino   
4   s7114    Movie                         Jackie Brown  Quentin Tarantino   
5   s7198    Movie                    Kill Bill: Vol. 1  Quentin Tarantino   
6   s7199    Movie                    Kill Bill: Vol. 2  Quentin Tarantino   
7   s7803    Movie                         Pulp Fiction  Quentin Tarantino   

                                                cast                 country  \
0  Jamie Foxx, Christoph Waltz, Leonardo DiCaprio...           United States   
1  Samuel L. Jackson, Kurt Russell, Jennifer Jaso...                    None   
2  Samue

In [48]:
# Problem 8: List all TV shows with more than 5 seasons
print("\n8. TV shows with more than 5 seasons:")
print(run_query("""
    SELECT 
        *
    FROM netflix
    WHERE 
       type = 'TV Show'
       AND
       CAST(SUBSTR(duration, 1, INSTR(duration, ' ') - 1) AS INTEGER) > 5
"""))


8. TV shows with more than 5 seasons:
   show_id     type                          title         director  \
0       s9  TV Show  The Great British Baking Show  Andy Devonshire   
1      s56  TV Show                      Nailed It             None   
2      s66  TV Show                   Numberblocks             None   
3      s68  TV Show              Saved by the Bell             None   
4      s83  TV Show                        Lucifer             None   
..     ...      ...                            ...              ...   
94   s8190  TV Show         The Andy Griffith Show             None   
95   s8379  TV Show                     The L Word             None   
96   s8443  TV Show              The Office (U.S.)             None   
97   s8558  TV Show                  The West Wing             None   
98   s8711  TV Show                          Weeds             None   

                                                 cast                country  \
0   Mel Giedroyc, Sue Perkin

In [50]:
# Problem 9: Count the number of content items in each genre
print("\n9. Content count by genre:")
print(run_query("""
    WITH split_genres AS (
        SELECT 
            show_id,
            TRIM(value) as genre
        FROM netflix, 
             json_each('["' || REPLACE(listed_in, ',', '","') || '"]')
        WHERE listed_in IS NOT NULL
    )
    SELECT
        genre,
        COUNT(show_id) as total_content
    FROM split_genres
    GROUP BY genre
    ORDER BY total_content DESC
"""))


9. Content count by genre:
                           genre  total_content
0           International Movies           2752
1                         Dramas           2427
2                       Comedies           1674
3         International TV Shows           1351
4                  Documentaries            869
5             Action & Adventure            859
6                      TV Dramas            763
7             Independent Movies            756
8       Children & Family Movies            641
9                Romantic Movies            616
10                   TV Comedies            581
11                     Thrillers            577
12                Crime TV Shows            470
13                      Kids' TV            451
14                    Docuseries            395
15              Music & Musicals            375
16             Romantic TV Shows            370
17                 Horror Movies            357
18               Stand-Up Comedy            343
19          

In [60]:
# Problem 10: Average content release per year in France
print("\n10. Average content release per year in France:")
print(run_query("""
    WITH France_content AS (
        SELECT * FROM netflix WHERE country LIKE '%France%'
    ),
    yearly_counts AS (
        SELECT 
            release_year as year,
            COUNT(*) as yearly_content
        FROM France_content
        GROUP BY release_year
    )
    SELECT 
        year,
        yearly_content,
        ROUND(
            yearly_content * 100.0 / (SELECT COUNT(*) FROM France_content), 
            2
        ) as percentage_of_total
    FROM yearly_counts
    ORDER BY yearly_content DESC
    LIMIT 5
"""))


10. Average content release per year in France:
   year  yearly_content  percentage_of_total
0  2016              52                13.23
1  2019              49                12.47
2  2018              47                11.96
3  2017              40                10.18
4  2015              33                 8.40


In [54]:
# Problem 11: List all documentaries
print("\n11. All documentaries:")
print(run_query("""
    SELECT * FROM netflix
    WHERE listed_in LIKE '%Documentaries%'
"""))


11. All documentaries:
    show_id   type                                              title  \
0        s1  Movie                               Dick Johnson Is Dead   
1       s17  Movie  Europe's Most Dangerous Man: Otto Skorzeny in ...   
2       s46  Movie                             My Heroes Were Cowboys   
3       s69  Movie                                         Schumacher   
4       s89  Movie           Blood Brothers: Malcolm X & Muhammad Ali   
..      ...    ...                                                ...   
864   s8740  Movie                 Why We Fight: The Battle of Russia   
865   s8745  Movie                                           Williams   
866   s8747  Movie                                             Winnie   
867   s8757  Movie                                          Woodstock   
868   s8764  Movie                    WWII: Report from the Aleutians   

                                              director  \
0                                      Ki

In [56]:
# Problem 12: Find all content without a director
print("\n12. Content without a director:")
print(run_query("""
    SELECT * FROM netflix
    WHERE director IS NULL
"""))


12. Content without a director:
     show_id     type                                title director  \
0         s2  TV Show                        Blood & Water     None   
1         s4  TV Show                Jailbirds New Orleans     None   
2         s5  TV Show                         Kota Factory     None   
3        s11  TV Show  Vendetta: Truth, Lies and The Mafia     None   
4        s15  TV Show      Crime Stories: India Detectives     None   
...      ...      ...                                  ...      ...   
2629   s8796  TV Show                      Yu-Gi-Oh! Arc-V     None   
2630   s8797  TV Show                           Yunus Emre     None   
2631   s8798  TV Show                            Zak Storm     None   
2632   s8801  TV Show                   Zindagi Gulzar Hai     None   
2633   s8804  TV Show                          Zombie Dumb     None   

                                                   cast  \
0     Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...

In [74]:
# Problem 13: Movies with Brad Pitt in last 10 years
print("\n13. Movies with Brad Pitt in last 10 years:")
current_year = datetime.now().year
print(run_query(f"""
    SELECT * FROM netflix
    WHERE 
        casts LIKE '%Brad Pitt%'
        AND 
        release_year > {current_year - 10}
"""))


13. Movies with Salman Khan in last 10 years (Fixed):


DatabaseError: Execution failed on sql '
    SELECT * FROM netflix
    WHERE 
        (casts LIKE '%Salman Khan%' OR casts LIKE '%Khan, Salman%')
        AND type = 'Movie'
        AND release_year > 2015
': no such column: casts

In [78]:
# Problem 14: Top 10 actors in French content
print("\n14. Top 10 actors in French content:")
print(run_query("""
    WITH split_actors AS (
        SELECT 
            show_id,
            TRIM(value) as actor
        FROM netflix, 
             json_each('["' || REPLACE(casts, ',', '","') || '"]')
        WHERE country LIKE '%French%' AND casts IS NOT NULL
    )
    SELECT
        actor,
        COUNT(*) as total_content
    FROM split_actors
    GROUP BY actor
    ORDER BY total_content DESC
    LIMIT 10
"""))


14. Top 10 actors in French content:


DatabaseError: Execution failed on sql '
    WITH split_actors AS (
        SELECT 
            show_id,
            TRIM(value) as actor
        FROM netflix, 
             json_each('["' || REPLACE(casts, ',', '","') || '"]')
        WHERE country LIKE '%French%' AND casts IS NOT NULL
    )
    SELECT
        actor,
        COUNT(*) as total_content
    FROM split_actors
    GROUP BY actor
    ORDER BY total_content DESC
    LIMIT 10
': no such column: casts

In [68]:
# Problem 15: Categorize content based on keywords in description
print("\n15. Content categorization based on keywords:")
print(run_query("""
    WITH categorized_content AS (
    SELECT 
        *,
        CASE 
            WHEN description LIKE '%kill%' OR
                 description LIKE '%violence%' THEN 'Bad_Content'
            ELSE 'Good_Content'
        END as category
    FROM netflix
    WHERE description IS NOT NULL
    )
    SELECT 
        category,
        COUNT(*) as total_content
    FROM categorized_content
    GROUP BY category
"""))


15. Content categorization based on keywords:
       category  total_content
0   Bad_Content            342
1  Good_Content           8465
